# Hands-on maskinlæringseksempel

I denne notebooken går vi gjennom et konkret eksempel på maskinlæring og viser stegene i prosessen fra rådata til maskinlæringsmodell. 

Notebooken er ikke ment som en innføring i maskinlæring. Maskinlæring kommer i mange former og fasonger, og her vil vi kun ta for oss ett enkelt eksempel på én type maskinlæring. Vi håper likevel at gjennomgangen kan bidra til å avmystisfisere maskinlæringsbegrepet litt og vise at det ikke nødvendigvis er så mye som skal til for å kunne dra nytte av maskinlæring.

Notebooken er strukturert som følger:
* Introduksjon: casebeskrivelse og kontekst
* Innhenting og sammenstilling av datasett
* Preprossesering av data for maskinlæring
* Trening av maskinlæringsmodell
* Evaluering av resultater

#### Grunnleggende notebook-kommandoer
For de som ikke kjenner Jupyter notebooks eller Google Colab fra før

* En notebook inneholder typisk en blanding av tekst-celler (slik som denne) og kode-celler.  I kodecellene er det kjørbar pythonkode. 

* Denne notebooken er det meningen at du skal følge steg for steg (celle for celle). Du kan bla deg nedover med piltasten, eller klikke på en ønsket celle med musen.

* For å kjøre en kode-celle kan du enten
>* markere den og trykke på play-symbolet til venstre for cellen
>* markere den og trykke *shift + enter*

In [0]:
# Eksempel på en kode-celle 
10 + 10

For flere hurtigtaster, tips og hjelp se: https://colab.research.google.com/notebooks/welcome.ipynb

## Casebeskrivelse: næringskodeklassifisering

Eksempelet vi skal ta for oss dreier seg om å bruke maskinlæring til næringskodeklassifisering.

 

**Hva er en næringskode?**

Ved registrering av en ny enhet i Enhetsregisteret tildeles enheten en (eller fler) næringskode(r). En næringskode er et femsiftet tall som beskriver en enhets hovedaktivitet. For eksempel er NAV registrert med næringskoden 84.300 - "Trygdeordninger underlagt offentlig forvaltning", mens Brønnøysundregistrene har koden 84.110  "Generell offentlig administrasjon". Næringskodene som benyttes i Norge bygger på EUs næringsstandard NACE.  


Næringskodene er beskrevet nærmere hos Brønnøysundregistrene: https://www.brreg.no/bedrift/naeringskoder/

En komplett oversikt over næringskodene finnes hos SSB:  https://www.ssb.no/klass/klassifikasjoner/6




**Næringskodeklassifisering**

Hensikten med næringskodeklassifiseringen er først og fremst å kunne bruke den til diverse rapportering og statistikk. Enhetsregisteret fastsetter i dag næringskoder manuelt basert på tekstlige beskrivelser av enhetenes formål. Dersom man hadde hatt en maskinlæringsmodell som kunne gjort jobben i stedet er potensielle gevinster at

*   Prosessen med å klassifisere nye enheter effektiviseres
*  Økt brukervennlighet for enhetsregisteret - forslag til passende næringskoder kan dukke opp automatisk når man fyller inn beskrivelse av virksomheten
*   Man kan anvende modellen på tidligere registrerte enheter også, og på den måten avdekke feilklassifiserte enheter 
*  Kvaliteten på statistikk og rapporter som benytter næringskoder heves

Vi ønsker altså å lage en modell som predikerer næringskoder basert på en tekstlig beskrivelse av en virksomhets formål. Med andre ord: vi trenger en tekstklassifiseringsmodell. Tekstklassifisering hører til den grenen av maskinlæring som kalles NLP (Natural Language Processing). Dette er et stort fagområde i seg selv, som omhandler hvordan vi kan lære datamaskiner å tolke naturlig språk (tale og tekst). https://en.wikipedia.org/wiki/Natural_language_processing

##  Datasett

For å trene opp en næringskodeklassifiseringsmodell trenger vi eksempler på ulike foretaks formålsbeskrivelser og deres tilhørende næringskoder.

Dette er informasjon som ligger åpent tilgjengelig på nett. Hos Brønnøysundregistrene kan vi finne følgende to lister:

* Næringskoder pr. foretak
* Formålsbeskrivelse pr. foretak

For å begrense tiden det tar å kjøre gjennom denne demoen har vi på forhånd laget subsett om kun inneholder 10 % av foretakene fra listene til Brønnøysundregistrene. Subsettene har vi lagt i en delt mappe på Google Drive. Vi vil i de neste stegene vise hvordan man laster ned disse subsettene.

Skulle du på et senere tidspunkt ha lyst til å laste ned de komplette datasettene direkte fra Brønnøysundregistrene finner du disse her:
* https://schema.brreg.no/filer2018/nacekoder.tab
* https://schema.brreg.no/filer2018/formaal.tab

La oss komme i gang!

### Koble til Google Drive og åpne delt mappe

Kjør sekvensen nedenfor for å koble til din Google Drive konto. Du må gjøre dette for å kunne få tilgang til den delte Drive-mappen med subsettene. 

Når du kjører kode-cellen vil du bli bedt om å fylle inn en autentiseringsnøkkel. Denne finner du ved å følge lenken som dukker opp i vinduet nedenfor. 

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Datasettene ligger i den delte mappen https://drive.google.com/open?id=1NLFB5bDfhuwf8CQcn5VWepqhRAeLpLnF. Vi han aksessere denne mappen via notebooken og printe ut en liste over filene den inneholder:

In [0]:
file_list = drive.ListFile({'q': "'1NLFB5bDfhuwf8CQcn5VWepqhRAeLpLnF' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s\t\t\t id: %s' % (file1['title'], file1['id']))

Etter å ha kjørt de to kode-cellene ovenfor har du alt du trenger for å kunne få tak i datasettene.

### Innlesing av data

Vi vil nå laste ned de to datasettene fra Google Drive. Til hjelp benytter vi python-biblioteket pandas. 

In [0]:
# Laster inn python-biblioteket pandas som vi vil benytte for dataanalyse og -manipulering 
import pandas as pd

#### Datasett 1: NACE (næringsgruppe) koder pr. foretak
Datasett 1 er en liste med organisasjonsnummer og tilhørende næringskoder. 

In [0]:
# Laster inn datasettet med næringskoder fra den delte mappen på Google Drive:
nace_download = drive.CreateFile({'id':file_list[2]['id']}) 
nace_download.GetContentFile(file_list[2]['title'])
nace = pd.read_csv('nacekoder_sample.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 

In [0]:
# Kontrollerer antall rader i datasettet:
len(nace) 

In [0]:
# Tar en titt på de fem første radene i datasettet:
nace.head(5)

Dersom en enhet har flere næringskoder vil enheten ha flere rader i datasettet (én pr. næringskode).
Vi ønsker å ha én rad pr. enhet, så vi flater ut datasettet og lager heller en kolonne pr. næringskode:

In [0]:
# Kodesnutt for å flate ut rader til kolonner:
nace_pivot = pd.pivot_table(nace.sort_values(by=['orgnr', 'rekkefolge']),
                            index='orgnr', columns='rekkefolge',
                            values='nacekode', aggfunc='first').reset_index()
nace_pivot.columns=['orgnr', 'nace_1', 'nace_2', 'nace_3'] 

In [0]:
# Ser på de fem første radene i datasettet etter pivotering:
nace_pivot.head(5)

In [0]:
# Sjekker antall rader i datasettet etter pivotering:
len(nace_pivot) 

#### Datasett 2: Formålet med virksomheten beskrevet i tekst
Dette er teksten som søker selv har skrevet for å få tildelt næringskode(r)

In [0]:
# Laster inn datasettet fra den delte mappen på Google Drive:
formaal_download = drive.CreateFile({'id':file_list[3]['id']})
formaal_download.GetContentFile(file_list[3]['title']) 
formaal = pd.read_csv('formaal_sample.tab', sep='\t', encoding='utf-8') 
# Ordner litt på formateringen:
formaal.drop(['tekst'], axis=1, inplace=True)
formaal.columns = ['orgnr', 'linje_nr', 'linje_tekst']

In [0]:
# Kontrollerer antall rader i datasettet:
len(formaal) 

In [0]:
# Ser på de fem første radene i datasettet:
formaal.head(5)

Noen ganger er det flere linjer med beskrivelse pr. orgnr. 

In [0]:
# Eksempel på orgnr. med beskrivelse over flere linjer:
list(formaal[formaal['orgnr'] == 811619752]['linje_tekst'])

Vi vil slå sammen disse slik at vi får én sammenhengende tekst pr. org_nr .

In [0]:
# Sammenslåing
formaal_merge = formaal[['orgnr','linje_tekst']].groupby('orgnr')['linje_tekst'].apply(lambda x: ' '.join(x)).reset_index()

In [0]:
# Eksempel etter sammenslåing
list(formaal_merge[formaal_merge['orgnr'] == 811619752]['linje_tekst'])

In [0]:
# Antall rader i datasettet etter sammenslåing:
len(formaal_merge)

In [0]:
# Ser på de fem første radene i datasettet etter sammenslåing:
formaal_merge.head()

### Koble sammen datasettene

Vi har nå to datasett:
* `nace_pivot` som inneholder organisasjonsnummer + næringskoder
* `formaal_merge` som inneholder organisasjonsnummer + formålstekst

Datasettene kan kobles sammen vha. organisasjonsnummeret:

In [0]:
# Sammenstiller formaal_merge og nace_pivot i et nytt datasett:
data = pd.merge(formaal_merge, nace_pivot, on='orgnr', how='inner') 

In [0]:
# Inspiserer resultatet
data.head(5)

In [0]:
# kontrollerer antall rader 
len(data) 

Nå har vi fått hentet inn og sammenstilt dataene slik vi ønsker.  Vanligvis bruker man en del tid på å utforske dataene for å bli bedre kjent med dem og å avdekke eventuelle rariteter. Vi viser ikke den prosessen her, men en et eksempel på en typisk ting man ville kunne oppdage er at noen formålsbeskrivelser kun inneholder ett tegn:

In [0]:
# Ser på noen tekster med med kun ett tegn i beskrivelsen:
data['lengde_linje_tekst'] = data['linje_tekst'].astype(str).apply(lambda x: len(x))
data[data['lengde_linje_tekst']<=1].head(5)

Dette gir ikke noen mening, så vi velger å fjerne de aller korteste fra datasettet: 

In [0]:
# Fjerner de korteste tekstene fra datasettet:
data = data[data['lengde_linje_tekst']>1]

In [0]:
# Antall gjenværende rader 
len(data)

## Dataprepp for maskinlæring

Selv om vi nå har et datasett som inneholder det vi trenger på et ryddig format, må tekstene typisk vaskes og re-formateres litt før vi kan fôre dem inn til en maskinlæringsalgoritme. Hva slags preprossesering som må gjøres avhenger både av hvordan dataene dine ser ut, hvilken algoritme du vil bruke og hva slags problem du skal løse. 

For vårt næringskodeklassifiseringsproblem finnes det utallige maskinlæringsalgoritmer vi kunne brukt. I denne demoen har vi valgt *fastText*, som er et open-source kodebibliotek som er utviklet av Facebook. 


Noen fordeler ved fastText:
* det går raskt å trene en tekstklassifiseringsmodell
* det krever relativt lite forarbeid
* man oppnår ofte gode resultater 

For mer informasjon og detaljer, se https://fasttext.cc/


### Formatering

**Generelle steg:**
*   Konverter alle tekstene til lower case: Dette gjør vi for å unngå at samme ord behandles ulikt bare på grunn av det er ulikt skrevet med store/små bokstaver.

*   Fjern alt som ikke er ord (tegnsetting). Det finnes mange metoder for å gjøre dette, men her har vi valgt en enkel og litt grov fremgangsmåte som fjerner alt av spesialtegn. Mer avanserte metoder vil kunne gi bedre resultater, men de tar også lenger tid å kjøre.

> Et eksempel på en mer avansert tilnærming er å benytte python-biblioteket *spacy* (https://spacy.io/). NAV har vært med på å utvikle en norsk språkmodell til spacy - hvis du er interessert kan du lese mer om denne her:  https://github.com/navikt/ai-lab-spacy-bokmaal. Ved å benytte denne metoden for å fjerne punktsetting vil maskinen for eksempel behandle forkortelsen "t.o.m." som noe annet enn ordet "tom", mens med vår enkle tilnærming fjernes alle punktum slik at "t.o.m." og "tom" vil se ut som samme ord.

*   Definer utfallsvariabel (dvs. hva modellen skal lære seg å predikere). Vi har i vårt datasett tre kolonner, `nace_1`, `nace_2` og `nace_3`. Vi velger å kun benytte `nace_1` som vår utfallsvariabel, siden den er utfylt for alle organisasjoner. Dersom en virksomhet har 2 eller 3 næringskoder bruker vi altså kun den første. 

**fastText-spesifikt:**
*  fastText krever en spesifikk struktur på datasettet. Inputen må være en liste av tekster der hver tekst også må inneholde utfallsvariabelen (i dette tilfellet næringskoden). For at fastText skal skjønne hva som er utfallsvariabelen må man i hver tekst markere den med prefikset \__label__ .

> Eksempel: 
En rad som ser slik ut

tekst| nace_1
--- | ---
skogbruk |    47.112
> i vårt datasett må transformeres til

> `'skogbruk   __label__47.112'`

In [0]:
# Lager en egen funksjon for å fikse tingene nevnt i teksten ovenfor
import re
from string import punctuation

regex = re.compile('[%s]' % re.escape(punctuation))

def get_dataset(row):
    s = row['linje_tekst']
    s = s.lower()
    s = regex.sub('', s)
    s += ' __label__' + str(row['nace_1'])
    return s

In [0]:
# Utfører datavasken og formateringen ved å anvende den egendefinerte funksjonen: 
dataset = data.apply(get_dataset, axis=1)

In [0]:
# Ser på resultatet
dataset.head(10)

### Inndeling i trening- og testsett

Det er vanlig praksis å evaluere en maskinlæringmodell ved å splitte datasettet sitt i to deler. Vi kaller den ene delen for et treningssett og den andre for et testsett eller valideringssett. Vi bruker treningssettet til å lære modellen sammenhenger mellom tekster og næringskoder, og så tester vi hvor bra modellen treffer med prediksjonene sine på testsettet.

Her gjør vi en splitt slik at 90% av datasettet går til trening og 10% til validering.

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.1)

In [0]:
# Lagrer del-datassettene
train.to_csv('næringskoder_train.txt', header = None, index = None, sep = ' ', encoding='utf-8')
test.to_csv('næringskoder_test.txt', header = None, index = None, sep = ' ', encoding='utf-8')

Koden ovenfor lagrer bare datasettene midlertidig på den virtuelle maskinen som Google Colab kjører på.

Dersom du ønsker å lagre datasettene varig kan du for eksempel lagre dem på Google Drive. Da må du i tillegg kjøre følgende snutter:

In [0]:
# For å lagre treningssettet på Google Drive:
upload_train = drive.CreateFile({'title': 'næringskoder_train.txt'})
upload_train.SetContentFile('næringskoder_train.txt')
upload_train.Upload()
drive.CreateFile({'id': upload_train.get('id')})

In [0]:
# For å lagre testsettet på Google Drive:
upload_test = drive.CreateFile({'title': 'næringskoder_test.txt'})
upload_test.SetContentFile('næringskoder_test.txt')
upload_test.Upload()
drive.CreateFile({'id': upload_test.get('id')})

Nå er vi klare for maskinlæring!

## Trene modell 

Som nevnt tidligere har vi her valgt å bruke fastText sin tekstklassifiseringsalgoritme.

fastText er ikke et standardbiblotek, så vi må laste det ned og innstallere det:

In [0]:
! git clone https://github.com/facebookresearch/fastText.git
% cd ./fastText
! pip install .
% cd /content/

In [0]:
# Når fastText er innstallert kan vi importere det som en vanlig python-modul:
import fastText

In [0]:
# Dette er koden for å trene opp en klassifiseringsmodell:
model = fastText.train_supervised(
    input = 'næringskoder_train.txt',
    wordNgrams = 3,
    lr = 1.0
)

In [0]:
fastText.train_supervised??

Gratulerer, du har nå bygget en tekstklassifiseringsmodell!

Merk at her satte vi parameterverdiene til modellen manuelt. Vanligvis kjører man modelltrening i flere iterasjoner og tester ulike parameterverdier. Hva som er optimale innstillinger vil variere avhengig av hvordan dataene man har ser ut og hvilket problem man forsøker å løse. 

For mer informasjon om parameterene som inngår i treningen av modellen og hvordan fastText virker, se for eksempel github: https://github.com/facebookresearch/fastText#text-classification



In [0]:
# Lagrer modellen (midlertidig, på den virtuelle maskinen):
model.save_model('nace_model.bin')

In [0]:
# For å lagre modellfilen på Google Drive:
uploaded = drive.CreateFile({'title': 'nace_model.bin'})
uploaded.SetContentFile('nace_model.bin')
uploaded.Upload()
drive.CreateFile({'id': uploaded.get('id')})

## Resultater

Vi evaluerer modellen på testsettet som ble holdt av tidligere. For å få ut resultatene på et lettleselig format skriver vi først vår egen print-funksjon:

In [0]:
def print_results(N,p,r):
    print("N\t\t" + str(N))
    print("Accuracy\t{:.3f}".format(p))

In [0]:
print_results(*model.test('næringskoder_test.txt'))

Her er


*   N = antall evaluerte tekster
*   Accuracy - modellens presisjon, dvs. andelen korrekte prediksjoner



#### Evaluering

Hva tenker du, ble dette en god eller dårlig modell? Ta i betraktning at det er 820 ulike næringskoder. 

Merk at i denne gjennomkjøringen har vi forsøkt å begrense tiden det tar å trene en modell så mye som mulig. Dette går naturlig nok ut over modellens kvalitet. For å lage en bedre modell, forsøk

1.   Mer data - bruk de fulle datasettene
2.   Test ut andre parameterverdier, for eksempel lengere trening (sett parameteren *epoch* til mer enn defaultverdien 5). 




Merk også at accuracy ikke sier alt om kvaliteten på en prediksjonsmodell. Under "Analyse av resultater" evaluerer vi modellens resultater mer detaljert.

#### Operasjonalisering

For å bruke modellen i praksis trenger vi å kunne sende inn én og én ny tekst og få predikerte næringskoder tilbake. Her lager vi derfor en funksjon som tar inn en tekst og returnerer de 5 mest sannsynlige tilhørende næringskodene:

In [0]:
def predict_nace(model, tekst, k):
    res = []
    pred = model.predict(tekst, k=k)
    for index, item in enumerate(pred[0]):
        res.append({'nace': item.replace('__label__', '').replace('"',''), 'prob': pred[1][index] })
    return res

Vi tester funksjonen vår på en tilfeldig utvalgt tekst:

In [0]:
tekst = "turer i skog og mark"
print(tekst)

In [0]:
out = predict_nace(model, tekst, 5)
out

Vi kan slå opp hva disse næringskodene egentlig er for noe ved å laste ned standardbeskrivelsene fra Brønnøysundregistrene:

In [0]:
# Henter inn liste med næringskoder og standardtekstene som tilhører kodene
nace_beskrivelse = pd.read_csv('https://schema.brreg.no/filer2018/nace_beskrivelse.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 
nace_beskrivelse.drop(['beskrivelse2'], axis=1, inplace=True)
nace_beskrivelse.columns = ['nace', 'tekst']

In [0]:
# Kontroll av antall næringskoder lest inn
len(nace_beskrivelse)

In [0]:
# Ser på næringskodene for de fem prediksjonene våre
nace_beskrivelse[(nace_beskrivelse['nace'] == out[0]['nace']) |
                 (nace_beskrivelse['nace'] == out[1]['nace']) |
                 (nace_beskrivelse['nace'] == out[2]['nace']) | 
                 (nace_beskrivelse['nace'] == out[3]['nace']) |
                 (nace_beskrivelse['nace'] == out[4]['nace'])]

### Detaljert analyse av resultater

I denne seksjonen viser vi fram noen eksempler på hva man kan gjøre for å evaluere modellen i mer detalj. 

Vi kan for eksempel se på:
* Hvor bra gjør modellen vår det hvis vi ser på treffsikkerhet blant topp 3 predikerte koder (fremfor kun nr.1)? 
* Hvor er modellen vår mest/minst treffsikker?
* Hvor ofte treffer den på næringshovedkoden (de to første sifrene i næringskoden)?
* Kan vi avdekke feilklassifiserte virksomheter, for eksempel ved å se på hvor modellen vår var mest selvsikker men likevel ikke fikk riktig?


Merk: disse analysene gir ikke alltid så mye mening når analysen gjennomføres på så små datamengder som vi har i denne demoen. Noen næringskoder finnes det svært få eksempler av, og dette gjør at vi egentlig ikke kan trekke noen konklusjon om  modellens treffsikkerhet på disse klassene. 

#### Accuracy for topp 3 prediksjoner

Her lager vi oss en dataframe som for hver tekst i testsettet viser oss faktisk næringskode og topp 3 predikerte koder fra modellen. Vi velger her å si at prediksjonen er 

*   korrekt dersom første prediksjon er riktig
*   delvis korrekt dersom prediksjon nr. 2 eller 3 er riktig
*   feil dersom ingen av de topp 3 prediksjonene er riktige



In [0]:
pred_list=[]
for index, row in enumerate(test.values):
    nace = row[-6:]
    line = row[:-16]
    length = len(line)
    if (length > 1):
        pred = predict_nace(model, line,3)
        pred_1 = pred[0]['nace'][-7:]
        prob_1 = pred[0]['prob']
        pred_2 = pred[1]['nace'][-7:]
        pred_3 = pred[2]['nace'][-7:]

        korrekt = 'Nei'
        if pred_1 == nace:
            korrekt = 'Ja'

        if nace in [pred_2,pred_3]:
            korrekt = 'Delvis'

        pred_list.append([index, nace, korrekt, pred_1, prob_1, pred_2, pred_3,line])
    else:
        print(row)
        
pred = pd.DataFrame(pred_list)
pred.columns=['index','nace','korrekt','pred_1','prob_1','pred_2','pred_3','tekst']

In [0]:
# Ser på resultattabellen
pred.head()

In [0]:
# Sjekker andel korrekte eller delvis korrekte prediksjoner 
len(pred[pred['korrekt'] != 'Nei'])/len(pred)

#### Mest og minst treffsikker

Her analyserer vi treffsikkerheten til modellen pr. næringskode .

In [0]:
# Grupperer testsett-resultatene pr. næringskode og korrekthetsgrad
resultat = pred[['index', 'nace','korrekt']].groupby(['nace','korrekt']).count().reset_index()
resultat.columns = ['nace', 'korrekt', 'antall'] # merk: ikke antall prediksjoner , man antall faktiske 
resultat.head()

In [0]:
# Flater ut resultattabellen pr. næringskode, utleder noen flere måltall og kobler på nacebeskrivelsene
pv = pd.pivot_table(resultat, index='nace', columns='korrekt').reset_index()
pv.columns = ['nace', 'delvis_korrekt','korrekt','feil']
pv.fillna(0, inplace = True)
pv['antall'] = pv['feil'] + pv['korrekt'] + pv['delvis_korrekt']
pv['andel_korrekt'] = 100 * pv['korrekt'] / pv['antall']
pv['andel_delvis_korrekt'] = 100 * (pv['korrekt'] + pv['delvis_korrekt']) / pv['antall']
pv['andel_feil'] = 100 * pv['feil'] / pv['antall']
pv.sort_values(by='antall', ascending = False, inplace=True) 
pv_t = pd.merge(pv,nace_beskrivelse, how='left', on='nace' )
pv_t = pv_t.rename(columns={'tekst':'nace_beskrivelse'})
pv_t.head(5)

In [0]:
# Skriver ut de næringskodene med høyest treffiskkerhet
pv_t.sort_values(by='andel_feil', ascending = True, inplace=True)
pv_t.head(5)

In [0]:
# ..og de med lavest treffsikkerhet
pv_t.tail(5)

Visualiseringer kan være til god hjelp for å danne seg et overblikksblide over resultatene:

In [0]:
# Installerer ggplot-biblioteket
! pip install ggplot
from ggplot import *

In [0]:
# Plotter antall mot andel korrekt (pr. næringskode)
ggplot(aes(x='antall', y='andel_korrekt', color='andel_delvis_korrekt'), data=pv_t) +\
    geom_point() +\
    theme_bw() +\
    xlab("Antall") +\
    ylab("Andel korrekt") +\
    ggtitle("Antall vs Andel korrekt")

In [0]:
# Plotter antall mot andel korrekt (pr. næringskode)
ggplot(aes(x='antall', y='andel_delvis_korrekt', color='andel_korrekt'), data=pv_t) +\
    geom_point() +\
    theme_bw() +\
    xlab("Antall") +\
    ylab("Andel delvis korrekt") +\
    ggtitle("Antall vs Andel delvis korrekt")

#### Hovedkoder

De to første sifrene av næringsgruppekoden kalles hovedkoden. Det kan være interessant å se hvor ofte modellen treffer riktig på den sammenlignet med den fulle femsifrede koden, samt å undesøke om det er noen hovedkoder der modellen gjør det spesielt bra eller dårlig.

**Accuracy**

In [0]:
pred_list=[]
for index, row in enumerate(test.values):
    nace = row[-6:-4]
    line = row[:-16]
    length = len(line)
    if (length > 1):
        pred_hoved = predict_nace(model, line,3)
        pred_1 = pred_hoved[0]['nace'][-7:-4]
        prob_1 = pred_hoved[0]['prob']
        pred_2 = pred_hoved[1]['nace'][-7:-4]
        pred_3 = pred_hoved[2]['nace'][-7:-4]
        korrekt = 'Nei'
        if pred_1 == nace:
            korrekt = 'Ja'
        if (nace in [pred_2,pred_3] and nace not in pred_1):
            korrekt = 'Delvis'
        pred_list.append([index, nace, korrekt, pred_1, prob_1, pred_2, pred_3,line])
    else:
        print(row)
        
pred_hoved = pd.DataFrame(pred_list)
pred_hoved.columns=['index','nace','korrekt','pred_1','prob_1','pred_2','pred_3','tekst']

In [0]:
pred_hoved.head()

In [0]:
# Hovedkode accuracy
len(pred_hoved[pred_hoved['korrekt'] == 'Ja'])/len(pred_hoved)

In [0]:
# Hovedkode accuracy blant topp 3 prediksjoner ("delvis korrekt")
len(pred_hoved[pred_hoved['korrekt'] != 'Nei'])/len(pred_hoved)

**Mest/minst treffsikker**

In [0]:
# Skiller ut hovedkoden fra den femsifrede koden
pv_t['nace_hoved'] = pv_t['nace'].apply(lambda x: x[0:2])

In [0]:
pvh = pv_t[['nace_hoved','antall','korrekt','delvis_korrekt','feil']].groupby(['nace_hoved']).sum().reset_index()
pvh = pvh[pvh['antall']>0]
pvh['andel_feil'] = 100*pvh['feil']/pvh['antall']

In [0]:
# Mest treffsikre hovedkoder
pvh.sort_values(by='andel_feil', ascending = True, inplace=True)
pvh.head(5)

In [0]:
# Minst treffsikre næringskoder
pvh.tail(5)

#### Feilklassifiserte enheter?

For å avdekke mulige feilklassifiseringer ser vi på hvilke prediksjoner om modellen er mest sikker på, men som likevel blir rapportert som feil.

In [0]:
# Vi kobler på beskrivelsene av alle nacekodene, både de predikerte og de faktiske
res = pd.merge(pred,nace_beskrivelse, how='left', left_on='nace', right_on='nace', suffixes=['','_nace'])
res = pd.merge(res,nace_beskrivelse, how='left', left_on='pred_1', right_on='nace', suffixes=['','_pred_1'])
res = pd.merge(res,nace_beskrivelse, how='left', left_on='pred_2', right_on='nace', suffixes=['','_pred_2'])
res = pd.merge(res,nace_beskrivelse, how='left', left_on='pred_3', right_on='nace', suffixes=['','_pred_3'])

In [0]:
res.head()

In [0]:
# Ser på de topp 5 prediksjonene modellen var mest sikker på som ikke stemte
res[res['korrekt'] != 'Ja'].sort_values(by = 'prob_1', ascending = False).head()